In [ ]:
def Modelo_Regresion_Lineal(Variables_X, Variable_Y, Nombre_Variable):

    """
    Ajusta un modelo de regresión lineal y devuelve todos los estadísticos.
    
    """
    
    print(f"\nAjustando modelo para: {Nombre_Variable}")
    print("="*50)
    
    # Agregar constante para el intercepto.
    Variables_X_Con_Constante = sm.add_constant(Variables_X)
    
    # Ajustar el modelo.
    Modelo = sm.OLS(Variable_Y, Variables_X_Con_Constante).fit()
    
    # Extraer resultados principales.
    Resultados = {
        'Variable_Dependiente': Nombre_Variable,
        'N_Observaciones': len(Variables_X),
        'R_Cuadrado': Modelo.rsquared,
        'R_Cuadrado_Ajustado': Modelo.rsquared_adj,
        'F_Estadistico': Modelo.fvalue,
        'P_Valor_Modelo': Modelo.f_pvalue,
        'AIC': Modelo.aic,
        'BIC': Modelo.bic
    }
    
    # Extraer coeficientes y p-valores.
    Coeficientes = Modelo.params.to_dict()
    P_Valores = Modelo.pvalues.to_dict()
    Intervalos_Confianza = Modelo.conf_int()
    
    # Identificar variables significativas (p < 0.05).
    Variables_Significativas = []
    for Variable, P_Valor in P_Valores.items():
        if P_Valor < 0.05 and Variable != 'const':
            Variables_Significativas.append(Variable)
    
    # Mostrar resumen del modelo.
    print(f"N observaciones: {Resultados['N_Observaciones']}")
    print(f"R²: {Resultados['R_Cuadrado']:.4f}")
    print(f"R² ajustado: {Resultados['R_Cuadrado_Ajustado']:.4f}")
    print(f"F-estadístico: {Resultados['F_Estadistico']:.2f}")
    print(f"P-valor del modelo: {Resultados['P_Valor_Modelo']:.6f}")
    print(f"Variables significativas: {len(Variables_Significativas)}")
    
    # Mostrar variables significativas.
    if Variables_Significativas:
        print(f"\nVariables significativas (p < 0.05):")
        for Variable in Variables_Significativas[:10]:  # Mostrar primeras 10.
            Coef = Coeficientes[Variable]
            P_Val = P_Valores[Variable]
            IC_Inf = Intervalos_Confianza.loc[Variable, 0]
            IC_Sup = Intervalos_Confianza.loc[Variable, 1]
            print(f"  {Variable:<30} β = {Coef:8.4f}, p = {P_Val:.6f}, IC: [{IC_Inf:6.3f}, {IC_Sup:6.3f}]")
        
        if len(Variables_Significativas) > 10:
            print(f"  ... y {len(Variables_Significativas) - 10} variables más")
    else:
        print("No hay variables significativas (p < 0.05)")
    
    # Guardar todos los resultados.
    Resultados['Coeficientes'] = Coeficientes
    Resultados['P_Valores'] = P_Valores
    Resultados['Intervalos_Confianza'] = Intervalos_Confianza.to_dict()
    Resultados['Variables_Significativas'] = Variables_Significativas
    Resultados['Modelo_Objeto'] = Modelo
    
    return Resultados

In [ ]:
def Evaluar_Modelo_Regresion_Lineal(Variables_X, Variable_Y, Resultado_Modelo):

    """
    Evalúa si el modelo es adecuado y qué problemas puede tener.
    
    """
    
    print("DIAGNÓSTICO DEL MODELO:")
    print("="*40)
    
    # 1. Verificar supuestos de regresión lineal.
    Modelo = Resultado_Modelo['Modelo_Objeto']
    Residuos = Modelo.resid
    Valores_Ajustados = Modelo.fittedvalues
    
    # Test de normalidad de residuos.
    from scipy.stats import shapiro, jarque_bera
    Stat_Shapiro, P_Normalidad = shapiro(Residuos.sample(min(5000, len(Residuos))))
    
    print(f"1. NORMALIDAD DE RESIDUOS:")
    print(f"   Shapiro-Wilk p-valor: {P_Normalidad:.6f}")
    if P_Normalidad < 0.05:
        print("   ❌ Residuos NO son normales")
    else:
        print("   ✅ Residuos son normales")
    
    # 2. Test de homocedasticidad.
    from statsmodels.stats.diagnostic import het_breuschpagan
    try:
        _, P_Homo, _, _ = het_breuschpagan(Residuos, Variables_X)
        print(f"\n2. HOMOCEDASTICIDAD:")
        print(f"   Breusch-Pagan p-valor: {P_Homo:.6f}")
        if P_Homo < 0.05:
            print("   ❌ HAY heterocedasticidad (varianza no constante)")
        else:
            print("   ✅ Homocedasticidad OK")
    except:
        print("\n2. HOMOCEDASTICIDAD: No se pudo calcular")
    
    # 3. Multicolinealidad.
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    
    print(f"\n3. MULTICOLINEALIDAD (VIF > 10 es problemático):")
    Variables_X_Con_Constante = sm.add_constant(Variables_X)
    VIF_Altos = []
    
    for i, Variable in enumerate(Variables_X.columns):
        try:
            VIF = variance_inflation_factor(Variables_X_Con_Constante.values, i+1)
            if VIF > 10:
                VIF_Altos.append((Variable, VIF))
                print(f"   ❌ {Variable}: VIF = {VIF:.2f}")
        except:
            continue
    
    if not VIF_Altos:
        print("   ✅ No hay multicolinealidad severa")
    
    # 4. Análisis de la variable dependiente.
    print(f"\n4. ANÁLISIS DE LA VARIABLE DEPENDIENTE:")
    print(f"   Media: {Variable_Y.mean():.3f}")
    print(f"   Desviación estándar: {Variable_Y.std():.3f}")
    print(f"   Valores únicos: {Variable_Y.nunique()}")
    print(f"   Rango: [{Variable_Y.min()}, {Variable_Y.max()}]")
    
    # Verificar si hay muchos ceros.
    Proporcion_Ceros = (Variable_Y == 0).mean()
    print(f"   Proporción de ceros: {Proporcion_Ceros:.1%}")
    
    if Proporcion_Ceros > 0.5:
        print("   ⚠️  Muchos ceros - considerar modelo Zero-Inflated")
    
    # 5. Recomendaciones.
    print(f"\n5. RECOMENDACIONES:")
    
    if Resultado_Modelo['R_Cuadrado'] < 0.10:
        print("   - R² muy bajo: las variables no explican bien la dependiente")
    
    if Resultado_Modelo['P_Valor_Modelo'] > 0.05:
        print("   - Modelo no significativo: revisar selección de variables")
    
    if len(Resultado_Modelo['Variables_Significativas']) < 5:
        print("   - Pocas variables significativas: posible sobreajuste")
    
    # 6. Sugerencias de mejora.
    print(f"\n6. POSIBLES MEJORAS:")
    print("   - Reducir número de variables independientes")
    print("   - Transformar variables (log, sqrt, etc.)")
    print("   - Considerar interacciones entre variables")
    print("   - Verificar outliers")
    
    if Proporcion_Ceros > 0.3:
        print("   - Considerar modelos para datos con exceso de ceros")